In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json


## Pre-processamento

In [0]:
df = pd.read_csv('USvideos.csv')

In [36]:
df.columns

Index(['video_id', 'trending_date', 'title', 'channel_title', 'category_id',
       'publish_time', 'tags', 'views', 'likes', 'dislikes', 'comment_count',
       'thumbnail_link', 'comments_disabled', 'ratings_disabled',
       'video_error_or_removed', 'description'],
      dtype='object')

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40949 entries, 0 to 40948
Data columns (total 16 columns):
video_id                  40949 non-null object
trending_date             40949 non-null object
title                     40949 non-null object
channel_title             40949 non-null object
category_id               40949 non-null int64
publish_time              40949 non-null object
tags                      40949 non-null object
views                     40949 non-null int64
likes                     40949 non-null int64
dislikes                  40949 non-null int64
comment_count             40949 non-null int64
thumbnail_link            40949 non-null object
comments_disabled         40949 non-null bool
ratings_disabled          40949 non-null bool
video_error_or_removed    40949 non-null bool
description               40379 non-null object
dtypes: bool(3), int64(5), object(8)
memory usage: 4.2+ MB


In [38]:
df.describe()

,category_id,views,likes,dislikes,comment_count
count,40949.000000,4.094900e+04,4.094900e+04,4.094900e+04,4.094900e+04
mean,19.972429,2.360785e+06,7.426670e+04,3.711401e+03,8.446804e+03
std,7.568327,7.394114e+06,2.288853e+05,2.902971e+04,3.743049e+04
min,1.000000,5.490000e+02,0.000000e+00,0.000000e+00,0.000000e+00
25%,17.000000,2.423290e+05,5.424000e+03,2.020000e+02,6.140000e+02
50%,24.000000,6.818610e+05,1.809100e+04,6.310000e+02,1.856000e+03
75%,25.000000,1.823157e+06,5.541700e+04,1.938000e+03,5.755000e+03
max,43.000000,2.252119e+08,5.613827e+06,1.674420e+06,1.361580e+06


In [0]:
def count_unique(x):
  for col in x.columns:
    print(f'{col}: {x[col].nunique()}')

In [40]:
count_unique(df)

video_id: 6351
trending_date: 205
title: 6455
channel_title: 2207
category_id: 16
publish_time: 6269
tags: 6055
views: 40478
likes: 29850
dislikes: 8516
comment_count: 13773
thumbnail_link: 6352
comments_disabled: 2
ratings_disabled: 2
video_error_or_removed: 2
description: 6901


In [41]:
# Drop columns with no important information in our analysis:
#thumbnail_link: link to small image which identify the video in the search results

df.drop(columns='thumbnail_link', inplace=True)
print(df.columns)

Index(['video_id', 'trending_date', 'title', 'channel_title', 'category_id',
       'publish_time', 'tags', 'views', 'likes', 'dislikes', 'comment_count',
       'comments_disabled', 'ratings_disabled', 'video_error_or_removed',
       'description'],
      dtype='object')


In [63]:
# Read json with category_id names and assign the names in new dataframe column 

import json

# open json file with category names
with open('US_category_id.json', 'r') as f:
  categ_us = json.load(f)

categ_dic = {}

for k in categ_us['items']:
  categ_dic[k['id']]= k['snippet']['title']

#print(categ_dic)

# Creating new column with category names
temp = df.category_id.copy()
temp = temp.astype('str')
df['category_name'] = temp.map(categ_dic)

#print(df.loc[:10, ['category_id', 'category_name']])


{'1': 'Film & Animation', '2': 'Autos & Vehicles', '10': 'Music', '15': 'Pets & Animals', '17': 'Sports', '18': 'Short Movies', '19': 'Travel & Events', '20': 'Gaming', '21': 'Videoblogging', '22': 'People & Blogs', '23': 'Comedy', '24': 'Entertainment', '25': 'News & Politics', '26': 'Howto & Style', '27': 'Education', '28': 'Science & Technology', '29': 'Nonprofits & Activism', '30': 'Movies', '31': 'Anime/Animation', '32': 'Action/Adventure', '33': 'Classics', '34': 'Comedy', '35': 'Documentary', '36': 'Drama', '37': 'Family', '38': 'Foreign', '39': 'Horror', '40': 'Sci-Fi/Fantasy', '41': 'Thriller', '42': 'Shorts', '43': 'Shows', '44': 'Trailers'}
    category_id         category_name
0            22        People & Blogs
1            24         Entertainment
2            23                Comedy
3            24         Entertainment
4            24         Entertainment
5            28  Science & Technology
6            24         Entertainment
7            28  Science & Technolog

In [0]:
# Processing the time feature

df.trending_date = pd.to_datetime(df.trending_date, format='%y.%d.%m')



In [75]:
temp = df.groupby(df.trending_date.dt.year)


print(temp)

In [67]:
type(df.trending_date[0])

str

In [64]:
df.trending_date.value_counts()[:20]

18.25.05    200
18.23.04    200
18.21.04    200
18.24.04    200
18.08.05    200
17.24.11    200
18.21.01    200
18.12.01    200
18.02.04    200
18.20.02    200
18.24.01    200
17.31.12    200
18.02.05    200
18.01.05    200
18.19.05    200
17.08.12    200
18.19.02    200
18.10.03    200
18.14.04    200
17.20.12    200
Name: trending_date, dtype: int64

In [0]:
df.video_id.unique()[:10]

array(['2kyS6SvSYSE', '1ZAPwfrtAFY', '5qpjK5DgCt4', 'puqaWrEC7tY',
       'd380meD0W0M', 'gHZ1Qz0KiKM', '39idVpFF7NQ', 'nc99ccSXST0',
       'jr9QtXwC9vc', 'TUmyygCMMGA'], dtype=object)

In [0]:
df.iloc[0,:]

video_id                                                        2kyS6SvSYSE
trending_date                                                      17.14.11
title                                    WE WANT TO TALK ABOUT OUR MARRIAGE
channel_title                                                  CaseyNeistat
category_id                                                              22
publish_time                                       2017-11-13T17:13:01.000Z
tags                                                        SHANtell martin
views                                                                748374
likes                                                                 57527
dislikes                                                               2966
comment_count                                                         15954
thumbnail_link               https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg
comments_disabled                                                     False
ratings_disa

In [0]:
df.iloc[1, :]

video_id                                                        1ZAPwfrtAFY
trending_date                                                      17.14.11
title                     The Trump Presidency: Last Week Tonight with J...
channel_title                                               LastWeekTonight
category_id                                                              24
publish_time                                       2017-11-13T07:30:00.000Z
tags                      last week tonight trump presidency|"last week ...
views                                                               2418783
likes                                                                 97185
dislikes                                                               6146
comment_count                                                         12703
thumbnail_link               https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg
comments_disabled                                                     False
ratings_disa

In [0]:
df.iloc[2, :]

video_id                                                        5qpjK5DgCt4
trending_date                                                      17.14.11
title                     Racist Superman | Rudy Mancuso, King Bach & Le...
channel_title                                                  Rudy Mancuso
category_id                                                              23
publish_time                                       2017-11-12T19:05:24.000Z
tags                      racist superman|"rudy"|"mancuso"|"king"|"bach"...
views                                                               3191434
likes                                                                146033
dislikes                                                               5339
comment_count                                                          8181
thumbnail_link               https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg
comments_disabled                                                     False
ratings_disa